In [ ]:
import numpy as np # linear algebra
import pandas as pd
import os
from tqdm import tqdm
import shutil
from keras.preprocessing import image
import os
import json
import math
from tqdm import tqdm 

import cv2
import numpy as np
import pandas as pd
from keras import layers, optimizers
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential
import cv2
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
test = pd.read_csv('../input/shopee-product-matching/test.csv')
sample = pd.read_csv('../input/shopee-product-matching/sample_submission.csv')

In [ ]:
img_path = '../input/shopee-product-matching/train_images'
def get_image_array(file, show=False):
    
    img = image.load_img(os.path.join(img_path,file), target_size=(128, 128))
    img_tensor =image.img_to_array(img)                    # (height, width, channels)
    #img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]


    return img_tensor

In [ ]:
# img_path = '../input/shopee-product-matching/train_images'
# #     testimg=load_image(os.path.join(img_path,imag))

In [ ]:
dataset_array=[]
for file in tqdm(train['image'][0:5]): #先取五個當範例
    image_array=get_image_array(file)
    dataset_array.append(image_array)

In [ ]:
dataset_array = np.array(dataset_array)

In [ ]:
dataset_labels = np.array(train['label_group'][0:5])

In [ ]:
np.save("train_dataset_array.npy", dataset_array)
np.save("train_dataset_array_labels.npy", dataset_labels)   

In [ ]:
from tensorflow.keras.models import load_model

base_model = load_model('../input/model50e/MobileNet_TransferLearning.h5')
#base_model.summary()

In [ ]:
base_model.summary()

In [ ]:
model2 = Model(inputs=base_model.input, outputs=base_model.get_layer('global_average_pooling2d').output)

#train_dataset_array = dataset_array

train_features = model2.predict(dataset_array)
base_model = np.save('MobileNet_Train_Features.npy', train_features)
print(train_features.shape)

In [ ]:
import numpy
from sklearn.neural_network import MLPClassifier
import joblib


train_dataset_array_labels = dataset_labels
#print("Train Features Shape", train_features.shape)

classifier = MLPClassifier(solver='adam', hidden_layer_sizes=(500, 150), max_iter=20000)

classifier.fit(train_features, train_dataset_array_labels)

joblib.dump(classifier, 'MobileNet_TransferLearning.joblib')

train_predictions = classifier.predict(train_features)

train_correct_predictions = np.array(np.where(train_predictions == train_dataset_array_labels))
train_accuracy = np.round((train_correct_predictions.shape[1]/train_features.shape[0])*100, 2)
print("Train Accuracy : ", train_accuracy)

In [ ]:
train_predictions

In [ ]:
x= classifier.predict(train_features[0])

In [ ]:
# import numpy
# import keras
# import os
# import tensorflow as tf

# def images_to_array(dataset_dir, image_size):
#     dataset_array = []
#     dataset_labels = []

#     class_counter = 0

#     classes_names = os.listdir(dataset_dir)
#     for current_class_name in classes_names:
#         class_dir = os.path.join(dataset_dir, current_class_name)
#         images_in_class = os.listdir(class_dir)

#         #print("Class index", class_counter, ", ", current_class_name, ":" , len(images_in_class))

#         for image_file in images_in_class:
#             image_file_dir = os.path.join(class_dir, image_file)
#             img = keras.preprocessing.image.load_img(image_file_dir, target_size=(image_size, image_size))
#             img_array = keras.preprocessing.image.img_to_array(img)
#             img_array = img_array/255.0
#             dataset_array.append(img_array)
#             dataset_labels.append(class_counter)
#         class_counter = class_counter + 1
#     dataset_array = numpy.array(dataset_array)
#     dataset_labels = numpy.array(dataset_labels)
#     return dataset_array, dataset_labels



# train_dir = '../input/test1/kaggle/working/class/'
# image_size = 224
# train_dataset_array, train_dataset_array_labels = images_to_array(dataset_dir=train_dir, image_size=image_size)
# print("Training Data Array Shape :", train_dataset_array.shape)
# numpy.save("train_dataset_array.npy", train_dataset_array)
# numpy.save("train_dataset_array_labels.npy", train_dataset_array_labels)

# # test_dir = "/root/.keras/datasets/fruits-360/Test"
# # test_dataset_array, test_dataset_array_labels = images_to_array(dataset_dir=test_dir, image_size=image_size)
# # print("Test Data Array Shape :", test_dataset_array.shape)
# # numpy.save("test_dataset_array.npy", test_dataset_array)
# # numpy.save("test_dataset_array_labels.npy", test_dataset_array_labels)